Takes the iCalendar ICS as a CSV and parses out and presents the data in a nice, readable format.

In [40]:
import csv

csv_file = open("ics.csv")

In [41]:
csvreader = csv.reader(csv_file)

In [42]:
import re

In [43]:
# I'm thinking we step through the rows one at a time. 
running = True
events = []
current_event = {}

while running:
    row = next(csvreader)
    row_code = row[0]

    if row_code == "DTSTART": # note that the times aren't timezone-adjusted and I'll have to do that
        timestamp_match = re.match(r"VALUE=DATE-TIME:(?P<year>\d{4})(?P<month>\d{2})(?P<day>\d{2})T(?P<hour>\d{2})(?P<minute>\d{2})(?P<second>\d{2})", row[1])
        # subtract 4 hours from the time
        datetime_string = "" + str(int(timestamp_match.group("hour")) - 4) + ":" + timestamp_match.group("minute") + "\t" + timestamp_match.group("day") + "/" + timestamp_match.group("month") + "/" + timestamp_match.group("year")
        current_event["Start Time"] = datetime_string
    elif row_code == "DTEND": # note that the times aren't timezone-adjusted and I'll have to do that
        timestamp_match = re.match(r"VALUE=DATE-TIME:(?P<year>\d{4})(?P<month>\d{2})(?P<day>\d{2})T(?P<hour>\d{2})(?P<minute>\d{2})(?P<second>\d{2})", row[1])
        # subtract 4 hours from the time
        datetime_string = "" + str(int(timestamp_match.group("hour")) - 4) + ":" + timestamp_match.group("minute") + "\t" + timestamp_match.group("day") + "/" + timestamp_match.group("month") + "/" + timestamp_match.group("year")
        current_event["End Time"] = datetime_string
    elif "SUMMARY" in row_code:
        current_event["Summary"] = re.match(r"SUMMARY:(?P<first>.+)", row[0]).group("first")
    elif "DESCRIPTION" in row_code:
        first_col = re.match(r"DESCRIPTION:(?P<first>.+)", row[0])
        current_event["Description"] = first_col.group("first") + row[1] + row[2] + row[3] + row[4]
    elif row_code == "END:VEVENT":
        events.append(current_event)
        current_event = {}
    elif row_code == "END:VCALENDAR":
        running = False
print(events)

[{'Start Time': '14:40\t01/06/2022', 'End Time': '14:55\t01/06/2022', 'Summary': 'Outbound call 90meter Inc.', 'Description': "Spoke to Matt Vanselow.\\nsells a lot to government middleware for classification onto a classified network. if you have sipper?. Main product is Smartcard manager / card issuance workstation. PKI? Middleware (also have non-classified stuff). We've been bidding a lot lately so they noticed us. Will receive an email from him with product info expected to reply with info on possibly talking more and maybe meeting in CA."}, {'Start Time': '16:05\t01/06/2022', 'End Time': '16:06\t01/06/2022', 'Summary': 'Outbound call A Cloud Guru', 'Description': 'No response'}, {'Start Time': '16:10\t01/06/2022', 'End Time': '16:20\t01/06/2022', 'Summary': 'Outbound call AbacusNEXT', 'Description': "Spoke to Scott Meyers.\\nGovernment-specific: HotDocs. Takes Word documents and makes templates out of them. Insurance and banking / governments. Good for filling out the same documen

In [44]:
output_string = ""
for event in events:
    output_string = output_string + "Start: " + event["Start Time"] + "\t" + "End: " + event["End Time"] + "\n"
    company = re.match(r"Outbound call (?P<company>.+)", event["Summary"]).group("company")
    output_string = output_string + "Called " + company + "\n"
    output_string = output_string + event["Description"] + "\n\n"

In [45]:
output_string = output_string.replace(r'\\n', r'\n')

In [46]:
# output_string

In [47]:
print(output_string.encode('utf-8').decode('unicode_escape'))

Start: 14:40	01/06/2022	End: 14:55	01/06/2022
Called 90meter Inc.
Spoke to Matt Vanselow.
sells a lot to government middleware for classification onto a classified network. if you have sipper?. Main product is Smartcard manager / card issuance workstation. PKI? Middleware (also have non-classified stuff). We've been bidding a lot lately so they noticed us. Will receive an email from him with product info expected to reply with info on possibly talking more and maybe meeting in CA.

Start: 16:05	01/06/2022	End: 16:06	01/06/2022
Called A Cloud Guru
No response

Start: 16:10	01/06/2022	End: 16:20	01/06/2022
Called AbacusNEXT
Spoke to Scott Meyers.
Government-specific: HotDocs. Takes Word documents and makes templates out of them. Insurance and banking / governments. Good for filling out the same documents on a regular basis. Easy to change and update. Best for formal legal-esque stuff. Doesn't have a partner program or anything. Familiar with us. He's normally the guy we'd work with. Shaz